In [8]:
import login
import psycopg2
try:
    conn = psycopg2.connect(user = login.user,
                                  password = login.password,
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "cookingapp")

    cur = conn.cursor()
    print ( conn.get_dsn_parameters(),"\n")
    cur.execute("SELECT version();")
    record = cur.fetchone()
    print("You are connected to - ", record,"\n")
except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

{'user': 'cook', 'dbname': 'cookingapp', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 12.0, compiled by Visual C++ build 1914, 64-bit',) 



In [9]:
import pandas as pd
import numpy as np

Postup:
- vložení vybraných kategorií a požadovaných ingrediencí do pomocných tabulek (parametrizace dotazů)
- ve view navrzene_recepty se objeví recepty daných kategorií, které mají některé ingredience společné
- vypsat podle dalších podmínek (rating, fat, protein, ...)
- seřadit podle vhodného kritéria
$$
\kappa_1 = \alpha \frac{\mathit{shoda}}{\mathit{zvolených}}+\beta \cfrac{\mathit{shoda}}{\mathit{v\_receptu}}
$$
$$
\kappa_2 = \bigg(\frac{\mathit{shoda}}{\mathit{zvolených}}\bigg)^\alpha\cdot\bigg(\frac{\mathit{shoda}}{\mathit{v\_receptu}}\bigg)^\beta
$$

In [10]:
zvolene_kategorie = ["Vegetarian"]
zvolene_ingredience = ["butter", "bacon"]

sql1 = "INSERT INTO zvolene_kategorie(idk) VALUES ((SELECT kid FROM kategorie WHERE nazev = %s))"
sql2 = "INSERT INTO zvolene_ingredience(idi) VALUES ((SELECT iid FROM ingredience_pref WHERE nazev = %s))"

try:
    cur.execute("DELETE FROM zvolene_kategorie")
    cur.execute("DELETE FROM zvolene_ingredience")
    cur.executemany(sql1, [[x] for x in zvolene_kategorie])
    cur.executemany(sql2, [[x] for x in zvolene_ingredience])
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
    conn.rollback()

In [11]:
sql = ("SELECT * from navrzene_recepty ORDER BY "
"(CAST(shoda AS DOUBLE PRECISION)/pocet_ingredienci*CAST(shoda AS DOUBLE PRECISION)/zvoleno_ingredienci) DESC")

df = pd.read_sql(sql, conn)
df.head()

,rid,shoda,zvoleno_ingredienci,pocet_ingredienci,calories,protein,rating,sodium,fat,title,description,date
0,12882,2,3,5,248.0,1.0,5.000,14.0,1.0,Apple and Blackberry Sorbet,No ice cream maker is needed to make this flav...,2004-08-20 04:00:00
1,9349,2,3,6,286.0,5.0,1.875,68.0,1.0,Apple and Blackberry Meringue Souffles,Can be prepared in 45 minutes or less.,2004-08-20 04:00:00
2,3827,2,3,7,NaN,NaN,4.375,NaN,NaN,Blackberry Hand Pies,Eating one of these individual pies is quite d...,2006-07-20 20:49:48
3,14528,1,3,2,172.0,2.0,0.000,14.0,1.0,Blackberry-Raspberry Sauce,None,2004-11-30 19:14:54
4,11671,1,3,2,122.0,0.0,3.750,1.0,0.0,Blackberry-Poblano Margarita,The luscious purple color of this drink is a s...,2010-04-13 04:00:00


In [13]:
cur.close()
conn.close()